In [1]:
# IMPORTS
import os, openai, random, pandas as pd
from nltk.tokenize import word_tokenize

# SECRET DECODER RING
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
# our 7 model texts
with open('models.txt', mode='r') as models_file:
    models = [line.rstrip() for line in models_file]

# our 200 topics
with open("topics-edited.txt", "r") as topics_file:
    text = topics_file.read()
    # strings = [line.rstrip() for line in another_file]
    # one_string = " ".join(strings)
    topics = word_tokenize(text)

## Build a Model of Conspiracy Theories

In [3]:
# This is not for ChatGPT but for us to provide better
# specifications. We will take these 7 models and convert them 
# into a single string below.
texts = []
for item in models:
    words = word_tokenize(item)
    texts.append(words)

lengths = [len(text) for text in texts]
print(lengths, sum(lengths)/len(lengths))

[117, 157, 159, 117] 137.5


In [10]:
print(texts[3])

['all', 'the', 'conspiracies', ',', 'no', 'matter', 'confirmed', 'or', 'debunked', ',', 'are', 'all', 'flying', 'on', 'the', 'internet', ',', 'and', 'many', 'people', 'already', 'are', 'aware', 'of', 'gov', 'invading', 'other', 'contries', 'with', 'tax', 'money', ',', 'using', 'hitman', 'clapse', 'other', 'govs', ',', 'spread', 'toxic', 'chem', 'or', 'vac', 'in', 'the', 'past', ',', 'illegally', 'surveil', 'on', 'citizen', ',', 'corruption', ',', 'police', 'brutality', ',', 'inside', 'trade', 'etc', '.', 'the', 'list', 'goes', 'on', 'and', 'on', ',', 'problem', 'is', 'people', 'today', 'are', 'so', 'numb', 'to', 'these', 'wrong', 'doing', 'that', 'more', 'seems', 'to', 'be', 'OK', 'with', 'them', '.', 'In', 'the', 'past', 'people', 'actually', 'do', 'stuff', 'fiercely', 'to', 'tear', 'down', 'the', 'situation', ',', 'like', 'french', 'revolution', ',', 'religious', 'reformation', 'war', ',', 'people', 'are', 'so', 'tamed', 'by', 'entertainment', 'today']


In [4]:
# We will use these instructions for all queries
instructions = """
"You are an assistant designed to create texts.
"""

def chatgptquery (query):
    completion = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        temperature = 1.0,
        top_p = 0.5,
        messages=[{"role": "assistant", "content": instructions},
                  {"role": "user", "content": query}])
    tokens = completion.usage.total_tokens
    reply_content = completion.choices[0].message.content
    # Get the seed, token usage, and the text
    print(tokens)
    print(reply_content)

In [6]:
# ChatGPT expects string input
models_ = "\n".join([f"{i}. {item}" for i, item in enumerate(models, 1)])

# And here's our query
query = f"""
I am going to provide you with 4 writing samples. 
I want you to copy the writing style and produce 
a text drawn from all four of the texts. Please make sure
your response is written in the style of the sample texts.

These are the 4 texts to use as models: {models_}
"""

chatgptquery(query)

863
Lately, I've been diving into various conspiracy theories, starting with the JFK assassination and moving on to other topics such as the Paul Is Dead theory and the control of medication in the United States. It's interesting to see how these theories have spread on the internet and how many people are aware of government wrongdoings such as invading other countries, using hitmen, spreading toxic chemicals or vaccines, and illegally surveilling citizens. However, it's disheartening to see how numb people have become to these issues and how little action is taken to address them. In fact, it seems like the government and corporations are gaining more control over our minds and bodies every day. Despite this, there are still those who are determined to uncover the truth and fight against the corruption.


In [7]:
prompt = random.sample(topics, 3)

query_b = f"""
Create a text in the style of those provided 
using the following words: {prompt}.
"""

chatgptquery(query_b)
print(prompt)

164
Just a few years ago, it was hard to imagine that a single cell phone could cost more than a thousand USD. But now, with the latest technology and features, it's not uncommon to see prices skyrocket. However, despite the high price tag, cell phones have become an essential part of our daily lives. From staying connected with loved ones to managing our work schedules, these devices have revolutionized the way we communicate. So, whether you're willing to spend a fortune or just looking for a budget-friendly option, there's a cell phone out there for everyone.
['just', 'usd', 'cell']


## Generate New Conspiracy Theories

In [ ]:
prompts = []
tokens_used = []
responses = []

for _ in range(10):

    prompt = random.sample(topics, 3)
    instructions = """
    "You are an assistant designed to create texts. 
    Users will paste in a string of words, 
    and you will respond with a cohesive text.
    """
    
    query = f"""
    Create a conspiracy theory of about 135 words 
    using the following words: {prompt}.
    """
    
    completion = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        temperature = 1.0,
        top_p = 0.5,
        messages=[
            {"role": "system", "content": instructions},
            {"role": "user", "content": query}
        ])
    
    # Get token usage (cost) and response:
    tokens = completion.usage.total_tokens
    reply = completion.choices[0].message.content
    
    prompts.append(prompt)
    tokens_used.append(tokens)
    responses.append(reply)

In [ ]:
responses = pd.DataFrame(
    {'prompt': prompts,
     'tokens': tokens_used,
     'response': responses,
    })
responses.head()

In [ ]:
# responses.to_csv("10-responses.csv")

## Using Only One Model

In [12]:
# Model below is from r/conspiracy
# 7th top post of the day at 2023-06-03-0800
# https://www.reddit.com/r/conspiracy/comments/13zamrr/\
# foreign_powers_are_destabilizing_the_us/
prompt = random.sample(topics, 3)

query_b = f"""
Create a text in the style of the text provided below
but change the topics to: {prompt}.

I believe the Chinese Communist Party (with the help of some Arab royalty) 
have successfully executed a culture-wide subversion campaign, accelerating 
and provoking already existing problems and creating new ones. I believe 
the elite are aware of some or all of this, and are hoarding wealth and 
luxury while we are trapped in a state of controlled demolition. I believe 
the purpose is to usher in a more authoritarian culture, like China. This 
will include one universal banking app, just like China, controlled by the 
government and used against you (i.e. low Social Credit Scores prohibit 
spending in some cases). I believe there is nothing we can do, and that the 
demoralization is part of the process. I believe in 3-5 years I won't even 
be able to share these thoughts. Please tell me what you think.
"""

chatgptquery(query_b)
print(prompt)

429
I believe that the stock market is being manipulated by a small group of wealthy individuals, causing instability and unpredictability for the average investor. I also believe that London is facing a cultural crisis, with gentrification and rising costs of living pushing out long-time residents and erasing the city's unique character. Additionally, I believe that Congress is failing to address the needs of the American people, instead prioritizing the interests of corporations and lobbyists. I fear that these issues are all part of a larger plan to consolidate power and control, with the elite hoarding resources and opportunities while the rest of us struggle to get by. I worry that in the near future, we will see the implementation of authoritarian policies, such as a universal banking system controlled by the government, limiting our freedoms and ability to live our lives as we choose. Despite these concerns, I remain hopeful that through collective action and awareness, we can w